<a href="https://colab.research.google.com/github/Yaswanthyarra/GAN-for-microexpression-augmentation/blob/main/CNN_GA_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_dir = '/content/drive/My Drive/CASMEII/Final_Apex_Frames_Segregated'
augmented_dir='/content/drive/My Drive/CASMEII/Augmented_Apex_Frames'

In [14]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split


dataset_dir = '/content/drive/My Drive/CASMEII/Final_Apex_Frames_Segregated'

label_dict = {'happiness': 0, 'disgust': 1, 'repression': 2, 'surprise': 3, 'fear': 4, 'others': 6, 'sadness': 5}

expression_images = []
expression_labels = []
# Initialize dictionaries to count instances for each label
label_counts = {}
for label in set([0,1,2,3,4,5,6]):
    label_counts[label] = 0

for class_name, label in label_dict.items():
    class_dir = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_dir)

    for image_file in images:
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)  # Load the image using OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        image = cv2.resize(image, (64,64))
        # Apply histogram equalization
        image = cv2.equalizeHist(image)
        if label==1:
          expression_images.append(image)
          expression_labels.append(label)
          # Update the label count
          label_counts[label] += 1
        if label==6:
          expression_images.append(image)
          expression_labels.append(label)
          # Update the label count
          label_counts[label] += 1



        expression_images.append(image)
        expression_labels.append(label)
        # Update the label count
        label_counts[label] += 1
# Print the label counts

for label, count in label_counts.items():
    label_name = [key for key, value in label_dict.items() if value == label][0]
    print(f'Label {label_name}: {count} instances')

Label happiness: 32 instances
Label disgust: 126 instances
Label repression: 27 instances
Label surprise: 25 instances
Label fear: 2 instances
Label sadness: 7 instances
Label others: 198 instances


In [16]:
augmented_dir='/content/drive/My Drive/CASMEII/Augmented_Apex_Frames'
label_dict = {'happiness': 0, 'repression': 2, 'surprise': 3, 'fear': 4, 'sadness': 5}
# Initialize dictionaries to count instances for each label
label_counts = {}
for label in set([0,2,3,4,5]):
    label_counts[label] = 0

for class_name, label in label_dict.items():
    class_dir = os.path.join(augmented_dir, class_name)
    images = os.listdir(class_dir)

    for image_file in images:
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)  # Load the image using OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Apply histogram equalization
        image = cv2.equalizeHist(image)
        if label==0:
          expression_images.append(image)
          expression_labels.append(label)
          label_counts[label] += 1

        expression_images.append(image)
        expression_labels.append(label)
        label_counts[label] += 1
for label, count in label_counts.items():
    label_name = [key for key, value in label_dict.items() if value == label][0]
    print(f'Label {label_name}: {count} instances')

Label happiness: 128 instances
Label repression: 64 instances
Label surprise: 50 instances
Label fear: 60 instances
Label sadness: 63 instances


In [18]:

# Convert lists to NumPy arrays
expression_images = np.array(expression_images)
expression_labels = np.array(expression_labels)

# Check the shape of the image data and labels
print("Expression Images Shape:", expression_images.shape)
print("Expression Labels Shape:", expression_labels.shape)
# Obtain the number of class labels
num_classes = len(np.unique(expression_labels))

# Stratified Splitting of data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(expression_images, expression_labels, test_size=0.2, stratify=expression_labels, random_state=42)

Expression Images Shape: (1147, 64, 64)
Expression Labels Shape: (1147,)


In [19]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.29),
    Dense(num_classes, activation='softmax')
])

# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.0006)

# Compile the model
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [20]:

# Train the model
model.fit(train_images, train_labels, epochs=52, batch_size=17, validation_data=(test_images, test_labels))


Epoch 1/52
54/54 [==============================] - 10s 151ms/step - loss: 5.0617 - accuracy: 0.3413 - val_loss: 0.9890 - val_accuracy: 0.7870
Epoch 2/52
54/54 [==============================] - 6s 111ms/step - loss: 0.8225 - accuracy: 0.7110 - val_loss: 0.5050 - val_accuracy: 0.8696
Epoch 3/52
54/54 [==============================] - 8s 146ms/step - loss: 0.5707 - accuracy: 0.7928 - val_loss: 0.4549 - val_accuracy: 0.8435
Epoch 4/52
54/54 [==============================] - 6s 112ms/step - loss: 0.4803 - accuracy: 0.8212 - val_loss: 0.4111 - val_accuracy: 0.8609
Epoch 5/52
54/54 [==============================] - 8s 144ms/step - loss: 0.4213 - accuracy: 0.8462 - val_loss: 0.3580 - val_accuracy: 0.8783
Epoch 6/52
54/54 [==============================] - 6s 109ms/step - loss: 0.3678 - accuracy: 0.8571 - val_loss: 0.4284 - val_accuracy: 0.8652
Epoch 7/52
54/54 [==============================] - 8s 140ms/step - loss: 0.3297 - accuracy: 0.8757 - val_loss: 0.3570 - val_accuracy: 0.8696
Epoch

In [21]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('Test accuracy:', test_acc)

8/8 - 1s - loss: 0.5663 - accuracy: 0.9348 - 773ms/epoch - 97ms/step
Test accuracy: 0.9347826242446899


In [22]:
from sklearn.metrics import classification_report
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)



# Calculate precision, recall, F1-score, and support
report = classification_report(test_labels, predicted_labels)
print(report)

8/8 [==============================] - 1s 65ms/step
              precision    recall  f1-score   support

           0       0.98      0.93      0.96        58
           1       0.88      0.92      0.90        25
           2       1.00      0.94      0.97        31
           3       0.96      0.88      0.92        25
           4       1.00      1.00      1.00        24
           5       1.00      0.93      0.96        27
           6       0.79      0.95      0.86        40

    accuracy                           0.93       230
   macro avg       0.94      0.93      0.94       230
weighted avg       0.94      0.93      0.94       230

